In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
avisos = pd.read_csv('Predecir/avisos_detalles_train.csv')
postulantes = pd.read_csv('Predecir/postulantes_datos_train.csv')
postulaciones = pd.read_csv('Predecir/df_postulaciones.csv')
df_postulantes_vistas_original = pd.read_csv('Originales/fiuba_3_vistas.csv')
df_postulantes_vistas_nuevo = pd.read_csv('Nuevos hasta 15 abril/fiuba_3_vistas.csv')
vistas = pd.concat([df_postulantes_vistas_original, df_postulantes_vistas_nuevo])

In [3]:
vistas.drop_duplicates(inplace=True)

In [10]:
vistas.rename(columns={'idAviso':'idaviso'}, inplace=True)

In [8]:
del vistas['timestamp']

In [4]:
del avisos['Unnamed: 0']
del postulantes['Unnamed: 0']
del postulaciones['Unnamed: 0']

In [5]:
populist = ['tipo_de_trabajo', 'nivel_laboral', 'nombre_area', 'denominacion_empresa']

for i in populist:
    avisos_specified = avisos.loc[:, ['idaviso', i]]
    
    datos_postulaciones_avisos = pd.merge(postulaciones, avisos_specified, on=['idaviso'], how='inner')
    datos_postulaciones_avisos = datos_postulaciones_avisos\
                                .groupby(['idpostulante', i]).count()\
                                .reset_index()
            
    datos = datos_postulaciones_avisos.loc[:, ['idpostulante', 'idaviso']]\
                                    .groupby('idpostulante').max().reset_index()            

    datos_postulaciones_avisos = pd.merge(datos_postulaciones_avisos, datos, on=['idaviso', 'idpostulante'], how='inner')
    datos_postulaciones_avisos = datos_postulaciones_avisos.drop_duplicates('idpostulante')
    datos_postulaciones_avisos = datos_postulaciones_avisos.loc[:, ['idpostulante', i]]
    postulantes = pd.merge(postulantes, datos_postulaciones_avisos, on=['idpostulante'], how='left')
    postulantes = postulantes.rename(columns={i:i+'_popular_postu'})

In [11]:
for i in populist:
    avisos_specified = avisos.loc[:, ['idaviso', i]]
    
    datos_postulaciones_avisos = pd.merge(vistas, avisos_specified, on=['idaviso'], how='inner')
    datos_postulaciones_avisos = datos_postulaciones_avisos\
                                .groupby(['idpostulante', i]).count()\
                                .reset_index()
            
    datos = datos_postulaciones_avisos.loc[:, ['idpostulante', 'idaviso']]\
                                    .groupby('idpostulante').max().reset_index()            

    datos_postulaciones_avisos = pd.merge(datos_postulaciones_avisos, datos, on=['idaviso', 'idpostulante'], how='inner')
    datos_postulaciones_avisos = datos_postulaciones_avisos.drop_duplicates('idpostulante')
    datos_postulaciones_avisos = datos_postulaciones_avisos.loc[:, ['idpostulante', i]]
    postulantes = pd.merge(postulantes, datos_postulaciones_avisos, on=['idpostulante'], how='left')
    postulantes = postulantes.rename(columns={i:i+'_popular_vistas'})

In [12]:
postulantes.head()

,idpostulante,nombre,estado,sexo,edad,vistas_postulante,cant_dias_actividad,postu_postulante,tipo_de_trabajo_popular,nivel_laboral_popular,nombre_area_popular,denominacion_empresa_popular,tipo_de_trabajo_popular_postu,nivel_laboral_popular_postu,nombre_area_popular_postu,denominacion_empresa_popular_postu,tipo_de_trabajo_popular_vistas,nivel_laboral_popular_vistas,nombre_area_popular_vistas,denominacion_empresa_popular_vistas
0,NdJl,4.0,1.0,2,49.0,60.0,88.0,30.0,1.0,3.0,10.0,233.0,1.0,3.0,10.0,233.0,1.0,3.0,10.0,140.0
1,1d2B,3.0,1.0,2,42.0,4.0,7.0,4.0,1.0,1.0,5.0,38.0,1.0,1.0,5.0,38.0,NaN,NaN,NaN,NaN
2,NPBx,5.0,1.0,2,37.0,34.0,81.0,34.0,1.0,1.0,10.0,1698.0,1.0,1.0,10.0,1698.0,1.0,1.0,10.0,410.0
3,ZKl5,5.0,1.0,2,44.0,41.0,89.0,11.0,1.0,5.0,1.0,872.0,1.0,5.0,1.0,872.0,1.0,5.0,1.0,173.0
4,Y043Z,3.0,1.0,2,39.0,12.0,50.0,4.0,1.0,1.0,25.0,185.0,1.0,1.0,25.0,185.0,1.0,1.0,25.0,269.0


In [15]:
def fill_popu_vistas(row, name):
    if ((np.isnan(row[name+'_popular_postu'])==False)&(np.isnan(row[name+'_popular_vistas']))):
        return row[name+'_popular_postu']
    return row[name+'_popular_vistas']

In [16]:
for word in populist:
    postulantes[word+'_popular_vistas'] = postulantes.apply((lambda x: fill_popu_vistas(x, word)), axis=1)

In [17]:
postulantes.head()

,idpostulante,nombre,estado,sexo,edad,vistas_postulante,cant_dias_actividad,postu_postulante,tipo_de_trabajo_popular,nivel_laboral_popular,nombre_area_popular,denominacion_empresa_popular,tipo_de_trabajo_popular_postu,nivel_laboral_popular_postu,nombre_area_popular_postu,denominacion_empresa_popular_postu,tipo_de_trabajo_popular_vistas,nivel_laboral_popular_vistas,nombre_area_popular_vistas,denominacion_empresa_popular_vistas
0,NdJl,4.0,1.0,2,49.0,60.0,88.0,30.0,1.0,3.0,10.0,233.0,1.0,3.0,10.0,233.0,1.0,3.0,10.0,140.0
1,1d2B,3.0,1.0,2,42.0,4.0,7.0,4.0,1.0,1.0,5.0,38.0,1.0,1.0,5.0,38.0,1.0,1.0,5.0,38.0
2,NPBx,5.0,1.0,2,37.0,34.0,81.0,34.0,1.0,1.0,10.0,1698.0,1.0,1.0,10.0,1698.0,1.0,1.0,10.0,410.0
3,ZKl5,5.0,1.0,2,44.0,41.0,89.0,11.0,1.0,5.0,1.0,872.0,1.0,5.0,1.0,872.0,1.0,5.0,1.0,173.0
4,Y043Z,3.0,1.0,2,39.0,12.0,50.0,4.0,1.0,1.0,25.0,185.0,1.0,1.0,25.0,185.0,1.0,1.0,25.0,269.0


In [18]:
postulantes.to_csv('Predecir/postulantes_datos_train.csv')

In [33]:
populist = ['edad', 'nombre']

In [34]:
for i in populist:
    postulantes_specified = postulantes.loc[:, ['idpostulante', i]]
    datos_postulaciones = pd.merge(postulaciones, postulantes_specified, on=['idpostulante'], how='inner')
    datos_postulaciones = datos_postulaciones.groupby(['idaviso', i]).count().reset_index()

    datos = datos_postulaciones.loc[:, ['idpostulante', 'idaviso']]\
                            .groupby('idaviso').max().reset_index()  

    datos_postulaciones = pd.merge(datos_postulaciones, datos, on=['idaviso', 'idpostulante'], how='inner')
    datos_postulaciones = datos_postulaciones.drop_duplicates('idaviso')
    datos_postulaciones = datos_postulaciones.loc[:, ['idaviso', i]]
    avisos = pd.merge(avisos, datos_postulaciones, on=['idaviso'], how='left')
    avisos = avisos.rename(columns={i:i+'_popular_postu'})    

In [35]:
for i in populist:
    postulantes_specified = postulantes.loc[:, ['idpostulante', i]]
    datos_postulaciones = pd.merge(vistas, postulantes_specified, on=['idpostulante'], how='inner')
    datos_postulaciones = datos_postulaciones.groupby(['idaviso', i]).count().reset_index()

    datos = datos_postulaciones.loc[:, ['idpostulante', 'idaviso']]\
                            .groupby('idaviso').max().reset_index()  

    datos_postulaciones = pd.merge(datos_postulaciones, datos, on=['idaviso', 'idpostulante'], how='inner')
    datos_postulaciones = datos_postulaciones.drop_duplicates('idaviso')
    datos_postulaciones = datos_postulaciones.loc[:, ['idaviso', i]]
    avisos = pd.merge(avisos, datos_postulaciones, on=['idaviso'], how='left')
    avisos = avisos.rename(columns={i:i+'_popular_vistas'})  

In [36]:
for word in populist:
    avisos[word+'_popular_vistas'] = avisos.apply((lambda x: fill_popu_vistas(x, word)), axis=1)

In [37]:
avisos.head()

,idaviso,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa,web,crecimiento,capacitacion,clima laboral,multinacional,...,supervisor,online,postu_total,total_vistas,vistas_median,cant_dias,edad_popular_postu,nombre_popular_postu,edad_popular_vistas,nombre_popular_vistas
0,8725750,1,1,1,1,0,0,1,0,0,...,0,0.0,137.0,137.0,NaN,26.0,26.0,3.0,26.0,3.0
1,17903700,1,1,2,2,0,0,0,0,0,...,0,1.0,175.0,1935.0,2.0,51.0,30.0,2.0,36.0,3.0
2,1000150677,1,1,3,3,0,0,0,0,0,...,0,0.0,NaN,2.0,2.0,3.0,NaN,NaN,38.0,1.0
3,1000610287,1,1,3,4,0,1,1,0,1,...,0,1.0,197.0,394.0,2.0,83.0,27.0,1.0,50.0,1.0
4,1000872556,1,1,4,5,0,0,0,0,0,...,1,0.0,NaN,1.0,1.0,0.0,NaN,NaN,25.0,1.0


In [38]:
avisos.to_csv('Predecir/avisos_detalles_train.csv')